In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/retail-products-classification-2023/how_to_download.txt


In [2]:
!pip install wget

  Preparing metadata (setup.py) ... - done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=62d14ed9b22175f16155b12c85003e0db9e20764f174ccaf8a54331d7dcd5258
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [3]:
import os
import pandas as pd
import random
import zipfile

In [4]:
!python -m wget https://filr.hs-offenburg.de/filr/public-link/file-download/0dcf8b85882ae199018870bceddf437c/5426/-2254907074900866378/test_data.zip

100% [....................................................] 28153268 / 28153268
Saved under test_data.zip


In [5]:
!python -m wget https://zenodo.org/record/7869954/files/products_leaflets_256.zip?download=1

100% [................................................] 1171218652 / 1171218652
Saved under products_leaflets_256.zip


In [6]:
path_to_zip_file = 'products_leaflets_256.zip'
directory_to_extract_to = '.'
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [7]:
path_to_zip_file = 'test_data.zip'
directory_to_extract_to = '.'
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [8]:
import os
import numpy as np
from PIL import Image



# Verzeichnis mit den Trainingsdaten
train_directory = 'products_leaflets_256/train'

# Bildgröße und Anzahl der Klassen
image_size = (224, 224)
num_classes = len(os.listdir(train_directory))

# Liste für Bilder und Labels
images = []
labels = []

# Durchlaufe das Trainingsverzeichnis und lade die Bilder
for root, dirs, files in os.walk(train_directory):
    for file in files:
        if file.endswith('.jpg'):
            image_path = os.path.join(root, file)
            image = Image.open(image_path).convert('RGB')
            image = image.resize(image_size)
            image_array = np.array(image)
            images.append(image_array)
            labels.append(int(os.path.basename(root)))

# Konvertiere die Listen in Numpy-Arrays
images = np.array(images)
labels = np.array(labels)




In [9]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

#ResNet50 with defaul parameters
model = MobileNetV2()
model2 = model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


# Trainieren des Modells mit 10 durchläufen
model.fit(images, labels, batch_size=32, epochs=3, validation_split=0.1)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

14536120/14536120 [==============================] - 0s 0us/step
Epoch 1/3
936/936 [==============================] - 2314s 2s/step - loss: 1.8873 - accuracy: 0.6545 - val_loss: 16.3784 - val_accuracy: 0.0000e+00
Epoch 2/3
936/936 [==============================] - 2341s 3s/step - loss: 0.2993 - accuracy: 0.9250 - val_loss: 19.9636 - val_accuracy: 0.0000e+00
Epoch 3/3
936/936 [==============================] - 2297s 2s/step - loss: 0.2048 - accuracy: 0.9453 - val_loss: 21.6161 - val_accuracy: 0.0000e+00


In [10]:
# Liste für Bildnamen und Vorhersagen
image_names = []
label_predictions = []
test_directory = 'test_data'

# Durchlaufe das Testverzeichnis und lade die Bilder
for root, dirs, files in os.walk(test_directory):
    for file in files:
        if file.endswith('.jpg'):
            image_path = os.path.join(root, file)
            image = Image.open(image_path).convert('RGB')
            image = image.resize(image_size)
            image_array = np.array(image)
            image_names.append(file)
            label_prediction = model.predict(np.expand_dims(image_array, axis=0))
            label_predictions.append(np.argmax(label_prediction))

# Erstelle den DataFrame und speichere die Vorhersagen in einer CSV-Datei
df = pd.DataFrame(columns=['image_id', 'label'])
df['image_id'] = image_names
df['label'] = label_predictions
df.to_csv('submission.csv', index=False)

1/1 [==============================] - 0s 44ms/step


# Neuer Abschnitt